<a href="https://colab.research.google.com/github/rbawden/Tutoriel-Normalisation/blob/main/Tutoriel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explorations sur la normalisation du français moderne

## 1. Setup de l'environnement, téléchargement des fichiers, etc.

Installer les paquets python

In [ ]:
!pip install fairseq@git+git://github.com/pytorch/fairseq.git@5a75b079bf8911a327940c28794608e003a9fa52 
!pip install sentencepiece sacrebleu hydra-core omegaconf==2.0.5 gdown==4.2.0 

Télécharger les données et les modèles depuis Google Drive et les structurer dans les dossiers `data/`, `models/`

In [ ]:
!gdown https://drive.google.com/drive/folders/1h-qSnPBPZFZQ_kqWIBMhkkFS-6C2b10H?usp=sharing -O data-models --folder
!mv data-models/structure_files.sh ./; bash structure_files.sh

## 2. Préparation des données à normaliser

Fonctions pour (i) lire le contenu d'un fichier ligne par ligne et (ii) les lire depuis un fichier

In [ ]:
# lire un fichier ligne par ligne
def read_file(filename):
  list_sents = []
  with open(filename) as fp:
    for line in fp:
      list_sents.append(line.strip())
  return list_sents

# écrire une liste de phrases dans un fichier
def write_file(list_sents, filename):
    with open(filename, 'w') as fp:
        for sent in list_sents:
            fp.write(sent + '\n')

Lire le contenu du texte source à normaliser (`dev.src`) et le texte cible ('de référence'), c'est-à-dire le texte correctement normalisé (`dev.trg`)

In [ ]:
dev_src = read_file('data/dev.src')
dev_trg = read_file('data/dev.trg')

Visualiser le début des textes sources (src) et cibles (trg)

In [ ]:
for i in range(4):
    print('src = ', dev_src[i])
    print('trg = ', dev_trg[i])
    print('--')

Charger le modèle de segmentation en sous-mots (`bpe_joint_1000.model`)

In [ ]:
import sentencepiece
spm = sentencepiece.SentencePieceProcessor(model_file='data/bpe_joint_1000.model')

Appliquer le modèle de segmentation sur les données à normaliser

In [ ]:
dev_src_sp = spm.encode(dev_src, out_type=str)

Écrire le texte pre-traité dans un fichier `dev.sp.src`

In [ ]:
write_file([' '.join(phrase) for phrase in dev_src_sp], 'data/dev.sp.src')

Visualiser le début du fichier

In [ ]:
dev_src_sp[:2]

Définir une fonction pour détokeniser une liste de phrases (pour plus tard)

In [ ]:
def decode_sp(list_sents):
    return [''.join(sent).replace(' ', '').replace('▁', ' ').strip() for sent in list_sents]

Visualiser à quoi ressemble le texte détokenisé (Spoiler: il devrait ressembler au texte de départ)

In [ ]:
decode_sp(dev_src_sp[:5])

## 3. Appliquer le modèle de normalisation

Appliquer le modèle de normalisation sur le début des données pre-traitées (ça prend moins de temps pour tester que normaliser tout le texte)

Il y aura un message "UserWarning", mais vous pouvez l'ignorer - ce n'est pas grave.

Explications:
- `head -n 10` affiche les 10 premières phrases
- ces 10 premières lignes sont donné à fairseq-interactive
- le résultat va dans `data/dev.sp.norm.trg.10.output`

In [ ]:
!head -n 10 data/dev.sp.src | fairseq-interactive models/norm/ --source-lang src --target-lang trg --path models/norm/lstm_norm.pt > data/dev.sp.norm.trg.10.output

La sortie de fairseq-interactive donne quelque chose comme ceci:

```
S-0     ▁1 .
H-0     -0.00011481383990030736 ▁1 .
P-0     -0.0000 -0.0003 -0.0000
S-1     ▁1 . ▁Q V e ▁cette ▁prop ost ion , ▁qu ' vn ▁esp ace ▁est ▁v ui d é , ▁re p u gne ▁au ▁sens ▁comm un .
H-1     -0.039981111884117126   ▁1 . ▁Q U e ▁cette ▁prop ost ion , ▁qu ' un ▁esp ace ▁est ▁v ui d é , ▁rép u gne ▁au ▁sens ▁comm un .
P-1     -0.0000 -0.0000 -0.0043 -0.0632 -0.0006 -0.0000 -0.0001 -0.9353 -0.0001 -0.0012 -0.0000 0.0000 -0.0001 -0.0078 -0.0070 -0.0000 -0.0022 -0.1168 -0.0001 -0.0000 -0.0000 -0.0389 -0.0157 -0.0053 -0.0000 -0.0000 -0.0001 -0.0000 -0.0004 -0.0000
S-2     ▁1 . ▁Q V e ▁tous ▁les ▁cor p s ▁ont ▁re p u gn ance ▁à ▁se ▁se p are r ▁l ' vn ▁de ▁l ' autre , ▁& ▁ad m ettre ▁du ▁v ui de ▁dans ▁leur ▁in ter u al le ;
W-2     0.682   seconds
H-2     -0.019450930878520012   ▁1 . ▁Q U e ▁tous ▁les ▁cor p s ▁ont ▁rép u gn ance ▁à ▁se ▁s ép are r ▁l ' un ▁de ▁l ' autre , ▁et ▁ad m ettre ▁du ▁v ui de ▁dans ▁leur ▁in ter v és le ;
D-2     -0.019450930878520012   ▁1 . ▁Q U e ▁tous ▁les ▁cor p s ▁ont ▁rép u gn ance ▁à ▁se ▁s ép are r ▁l ' un ▁de ▁l ' autre , ▁et ▁ad m ettre ▁du ▁v ui de ▁dans ▁leur ▁in ter v és le ;
P-2     -0.0000 -0.0001 -0.0040 -0.1684 -0.0004 -0.0000 -0.0000 -0.0000 -0.0007 -0.0000 -0.0001 -0.1220 -0.0063 -0.0002 -0.0137 -0.0000 -0.0000 -0.0002 -0.0001 -0.0248 -0.0022 -0.0003 -0.0000 -0.0000 -0.0000 -0.0000 -0.0000 -0.0000 -0.0002 -0.0001 -0.0000 -0.0000 -0.0000 -0.0000 -0.0383 -0.0173 -0.0006 -0.0000 -0.0000 -0.0000 -0.0066 -0.0016 -0.4856 -0.0007 -0.0002 -0.0000
```

Les informations intéressantes pour l'exemple `i`:

- S-i: le texte source
- H-i: le score de l'hypothèse et l'hypothèse du modèle (c'est-à-dire la prédiction)
- P-i: les scores de chaque sous-token produit par le modèle

Fonction pour extraire l'hypothèse de ce fichier

In [ ]:
def extract_hypothesis(filename):
    outputs = []
    with open(filename) as fp:
        for line in fp:
            # seulement les lignes qui commencet par H- (pour Hypothèse)
            if 'H-' in line:
                # prendre la 3ème colonne (c'est-à-dire l'indice 2)
                outputs.append(line.strip().split('\t')[2])
    return outputs

Extraire les hypothèses du fichier produit

In [ ]:
dev_norm_10 = extract_hypothesis('data/dev.sp.norm.trg.10.output')
dev_norm_10[:3]

Post-traiter le texte avec la fonction précedemment définie (dé-segmenter)

In [ ]:
dev_norm_10_postproc = decode_sp(dev_norm_10)
dev_norm_10_postproc[:3]

Écrire le résultat dans un fichier

In [ ]:
write_file(dev_norm_10_postproc, 'data/dev.norm.10.trg')

## 4. Évaluation du résultat

- BLEU: le métrique d'évaluation le plus fréquemment utilisé en traduction automatique
- ChrF: CharacterF score (like BLEU but based on n-grams of characters)
- TER: translation edit rate

Attention : puisque nous avons seulement normalisé 10 phrases, il faut seulement comparer contre les 10 première phrases de référence. En réalité, il faudrait calculer ces scores sur un plus grand nombre de phrases.

In [ ]:
from sacrebleu.metrics import BLEU, CHRF, TER
bleu = BLEU()
bleu.corpus_score(dev_norm_10_postproc, [dev_trg[:10]])

In [ ]:
chrf = CHRF()
chrf.corpus_score(dev_norm_10_postproc, [dev_trg[:10]])

In [ ]:
ter = TER()
ter.corpus_score(dev_norm_10_postproc, [dev_trg[:10]])

Une évaluation plus adaptée : la précision au niveau de chaque mot

In [ ]:
import align

In [ ]:
# d'abord créer un fichier qui ne contient que les 10 première phrases du document cible
!head -n 10 data/dev.trg > data/dev.10.trg
align_dev_norm_10 = align.align('data/dev.10.trg', 'data/dev.norm.10.trg')

print(align_dev_norm_10)

Le résultat de l'alignement est une liste de phrases, où chaque mot de la phrase est comme suit:

- le mot tout seul s'il est pareil dans les deux textes (ex : `QUe`)
- le mot du premier document et le mot du deuxième document, séparé par ">" s'ils sont différents (ex : proposition>propostion)

In [ ]:
num_diff = 0
total = 0
for sentence in align_dev_norm_10:
    for word in sentence:
        if '>' in word:
            num_diff += 1
        total += 1
print('Accuracy = ' + str((total - num_diff)/total))

# 5. Tester le modèle de dénormalisation

### Preparer les données normalisées qui vont être dénormalisées

In [ ]:
dev_trg_sp = spm.encode(dev_trg, out_type=str) # tokenise the sentence into subtokens
decade_token = '▁<decade=162> ' # special token indicating the decade
write_file([' '.join([decade_token] + phrase) for phrase in dev_trg_sp], 'data/dev.sp.trg')

### Normaliser le texte

(10 premières phrases seulement. Vous pouvez faire plus de phrases en modifiant le 10. Vous pouvez tout normaliser en changeant `head -n 10` en `cat`.)

In [ ]:
!head -n 10 data/dev.sp.trg | fairseq-interactive models/denorm --source-lang trg --target-lang src --path models/denorm/lstm_denorm.pt > data/dev.sp.denorm.src.10.output

### Post-traiter la sortie du modèle

In [ ]:
dev_denorm_10 = extract_hypothesis('data/dev.sp.denorm.src.10.output')
dev_denorm_10_postproc = decode_sp(dev_denorm_10)
write_file(dev_denorm_10_postproc, 'data/dev.sp.denorm.10.src')
dev_denorm_10_postproc[:3]

Il y a pas mal d'étapes, donc pour faciliter le traitement, voici une fonction qui prend en entrée une liste de phrases et qui fait tout :

In [ ]:
def denormalise(sents, decade):
    assert int(decade) >=1600 and int(decade) < 1700, 'Your decade must be between 1600 and 1690'
    # generate temporary file
    filetmp = 'data/tmp_denorm.sp.trg.tmp'
    # preprocessing
    input_sp = spm.encode(sents, out_type=str)
    # add decade token to each sentence
    decade_token = '▁<decade=' + str(decade)[:3] + '>'
    input_sp_sents = [' '.join([decade_token] + sent) for sent in input_sp]
    write_file(input_sp_sents, filetmp)
    #print("preprocessed = ", input_sp_sents)
    # denormalisation
    !cat data/tmp_denorm.sp.trg.tmp | fairseq-interactive models/denorm --source-lang trg --target-lang src --path models/denorm/lstm_denorm.pt > data/tmp_denorm.sp.trg.output 2> /tmp/dev
    # postprocessing
    outputs = extract_hypothesis('data/tmp_denorm.sp.trg.output')
    outputs_postproc = decode_sp(outputs)
    return outputs_postproc

Et on peut la tester comme suit :

In [ ]:
print(denormalise(["Je ne savais pas qu'il faisait si beau."], 1640))
print(denormalise(["Je ne savais pas qu'il faisait si beau."], 1690))

Et voici une fonction similaire pour la normalisation :

In [ ]:
def normalise(sents):
    # generate temporary file
    filetmp = 'data/tmp_norm.sp.src.tmp'
    # preprocessing
    input_sp = spm.encode(sents, out_type=str)
    # add decade token to each sentence
    input_sp_sents = [' '.join(sent) for sent in input_sp]
    write_file(input_sp_sents, filetmp)
    #print("preprocessed = ", input_sp_sents)
    # denormalisation
    !cat data/tmp_norm.sp.src.tmp | fairseq-interactive models/norm --source-lang src --target-lang trg --path models/norm/lstm_norm.pt > data/tmp_norm.sp.src.output 2> /tmp/dev
    # postprocessing
    outputs = extract_hypothesis('data/tmp_norm.sp.src.output')
    outputs_postproc = decode_sp(outputs)
    return outputs_postproc

In [ ]:
normalise(["1. QVe cette propostion, qu'vn espace est vuidé, repugne au sens commun.",
          "Affectoit un mépris qui marquoit ſon eſtime,"])

# 6. Quelques extensions (y compris du code à faire 👩🏻‍💻🧑🏽‍💻)

### Appliquer le modèle sur le texte entier

Attention : normaliser le fichier entier prend environ 6 minutes en utilisant le GPU

Si vous voulez juste l'évaluer et l'analyser (sans refaire la normalisation), le fichier entièrement normalisé est disponible ici: `data/dev.norm.full.trg` 

Ou vous pouvez normaliser une partie du fichier seulement (comme avant mais avec plus de phrases que 10)

In [ ]:
dev_norm = normalise(dev_src)

In [ ]:
write_file(dev_norm, 'data/dev.norm.trg')

 Refaire l'évaluation sur le texte entier (pas juste sur les 10 premières phrases) : BLEU, ChrF, TER et exactitude. 

In [ ]:
# TODO

Vous pouvez refaire la normalisation pour le jeu de test maintenant (il se trouve dans `data/test.src`) et l'évaluer/l'analyser

La normalisation complète se trouve dans `data/test.norm.full.trg` si vous ne voulez pas attendre

In [ ]:
test_src = read_file('data/test.src')
test_norm = normalise(test_src)
#test_norm = read_file('data/test.norm.full.trg') # pour utiliser le texte déjà normalisé, décommenter cette ligner et commenter la ligne précédente
write_file(test_norm, 'data/test.norm.trg')

In [ ]:
# TODO

Vous pouvez aussi faire la même évaluation sur le texte source pour voir quels seraient les scores si on ne changeait rien

In [ ]:
# TODO

### Faire une analyse qualitative de résultats (sur le dev et/ou le test)

Complétez cette fonction pour afficher les différences les plus fréquentes.

(Le calcul des alignements est lent, donc vous pouvez tester avec moins de phrases aussi, surtout lorsque vous tester simplement votre fonction)

In [ ]:
!head -n 100 data/dev.trg > data/dev.100.trg
!head -n 100 data/dev.norm.full.trg > data/dev.norm.100.trg
alignments_dev = align.align('data/dev.100.trg', 'data/dev.norm.100.trg')
#alignments_dev = align.align('data/dev.trg', 'data/dev.norm.full.trg') # décommenter pour traiter le jeu de dev en entier


# get alignments
def print_most_frequent_diffs(alignments, show_n=10):
    # TODO
    return

print_most_frequent_diffs(alignments_dev)

### Faire une analyse qualitative des différences entre un texte source et cible pour visualiser les corresondences de normalisation.

Idéalement ceci se fait sur le jeu d'entraînement (`data/train.src` et `data/train.trg`), mais ça risque de prendre trop de temps pour ce TP, donc prenez une sous-partie des exemples (en utilisant `head` comme avant) !

- lire les deux fichiers
- appliquer l'alignement
- utiliser la fonction `print_most_frequent_diffs` (précédemment utilisée pour comparer la prédiction contre la référence)

In [ ]:
# TODO

### Créer un modèle baseline par règles. Comme ressources supplémentaires, vous avez un lexique de mots en français contemporain et quelques fonctions

Idées possibles:

- remplacer les caractères qui changent systématiquement en utilisant la fonction `replace(avant, après)` : `word = word.replace('ſ', 's')`
- remplacer les caractères en utilisant les expressions régulières (si vous les connaissez) : `word = word.replace('vn(e?)', r'un\1')`
- parcourir les mots de la phrase et si le mot n'apparaît pas dans le lexique, trouver le mot du lexique qui est le plus similaire. Quelques idées de fonctions de similarité:
  - la distance de levenshtein (la fonction est donnée ci-dessous)
  - une fonction plus simple qui compare le nombre de caractères en commun entre les deux mots (à faire)
  - il serait peut-être sage de normaliser cette dernière similarité par la longueur des mots


In [ ]:
# read a lexicon in the mlex format
import gzip
def read_lexicon(filename):
    words = []
    with gzip.open(filename, 'rt') as fp:
        for line in fp:
            words.append(line.split('\t')[0])
    return words

# calculate the levenshtein distance
def similarity_levenshtein(word1, word2):
    dist, matrix, backpointers = levenshtein('@' + word1, '@' + word2)
    return dist

# calculate the number of common characters between the two words
def similarity_common_chars(word1, word2):
    # TODO
    return

In [ ]:
lexicon = read_lexicon('data/lefff-3.4.mlex.gz')

In [ ]:
print('Le lexique contient ' + str(len(lexicon)) + ' entrées.')

In [ ]:
# function that normalises a sentence given a function that normalises a word
import utils
from importlib import reload
reload(utils)
def normalise_sent(sent, normalise_word_function):
    norm_sent = []
    # go through the sentence word by word (the tokenisation function is very approximate here!)
    for word in utils.basic_tokenise(sent).split():
        norm_sent.append(normalise_word_function(word))
    return utils.detokenise(' '.join(norm_sent))

In [ ]:
# function that returns the word itself
def return_word(word):
    return word

In [ ]:
# illustration of how this could work (using just the function that returns the original word)
normalise_word_function = return_word
normalise_sent("QVe cette propoſtion, qu'vn eſpace eſt vuidé", normalise_word_function)

In [ ]:
# function that replaces the long s
def replace_long_s(word):
    word = word.replace('ſ', 's')
    return word

Maintenant on peut tester avec une autre petite fonction qui ne fait que remplacer les ſ par s :

In [ ]:
normalise_word_function = replace_long_s
normalise_sent("QVe cette propoſtion, qu'vn eſpace eſt vuidé", normalise_word_function)

Vous pouvez créer une fonction qui contient plus de remplacements (comme `word.replace(before, after)`).

Parfois, un remplacement peut être contextuel, donc si vous connaissez les expressions régulières vous pouvez les utiliser aussi.


In [ ]:
import re
def replace_regex(word):
    word = word.replace('ſ', 's')
    word = re.sub("([Qq])v", r'\1u', word)
    word = re.sub("([Qq])V", r'\1U', word)
    word = re.sub("('?)vn(e?)", r'\1un\2', word)
    return word

normalise_word_function = replace_regex
normalise_sent("QVe cette propoſtion, qu'vn eſpace eſt vuidé", normalise_word_function)

In [ ]:
# TODO

### Entraîner un modèle de segmentation en sous-mots en avec le toolkit sentencepiece

Ce sera un modèle joint - c'est-à-dire qu'il est entraîné pour segmenter la langue source et cible et ça permet de faire des sous-mots qui peuvent être partagés pour les deux langues. C'est surtout bien pour les langues proches lexicalement.

La taille du vocabulaire ici est de 2000, mais ceci peut être changé. La taille du vocabulaire détermine combien on découpé le texte. Plus le vocabulaire est petit, plus le texte sera découpé, plus le vocabulaire est grand, moins le texte sera découpé (ça ressemblera plus à un découpage sur les blancs). Vous pouvez tester avec des tailles de vocabulaires différents.

In [ ]:
# concatenate the source and target sides of the training set to train a joint model (encourages lexical sharing between the units)
!cat data/train.src data/train.trg > data/all_train.src-trg
sentencepiece.SentencePieceTrainer.train(input='data/all_train.src-trg', 
                               model_prefix='data/bpe_joint_2000', 
                               vocab_size=2000)

Lire les trois jeux (train, dev, test) et appliquer les pré-traitements

In [ ]:
train_src = read_file('data/train.src')
train_trg = read_file('data/train.trg')
dev_src = read_file('data/dev.src')
dev_trg = read_file('data/dev.trg')
test_src = read_file('data/test.src')
test_trg = read_file('data/test.trg')

# load newly trained sentencepiece models
spm = sentencepiece.SentencePieceProcessor(model_file='data/bpe_joint_2000.model')

# apply sentencepiece to each of the datasets
train_src_sp = spm.encode(train_src, out_type=str)
train_trg_sp = spm.encode(train_trg, out_type=str)
dev_src_sp = spm.encode(dev_src, out_type=str)
dev_trg_sp = spm.encode(dev_trg, out_type=str)
test_src_sp = spm.encode(test_src, out_type=str)
test_trg_sp = spm.encode(test_trg, out_type=str)

# print out lengths (src and trg must be the same length for each type of set)
print(len(dev_src_sp), len(train_trg_sp))
print(len(dev_src_sp), len(dev_trg_sp))
print(len(test_src_sp), len(test_trg_sp))

# write them to file
write_file([' '.join(sent) for sent in train_src_sp], 'data/train.sp2000.src')
write_file([' '.join(sent) for sent in train_trg_sp], 'data/train.sp2000.trg')
write_file([' '.join(sent) for sent in dev_src_sp], 'data/dev.sp2000.src')
write_file([' '.join(sent) for sent in dev_trg_sp], 'data/dev.sp2000.trg')
write_file([' '.join(sent) for sent in test_src_sp], 'data/test.sp2000.src')
write_file([' '.join(sent) for sent in test_trg_sp], 'data/test.sp2000.trg')

### Entraîner un modèle récurrent (de type LSTM) avec fairseq 

Il faut d'abord binariser les données pour rendre l'utilisation de données plus efficace

In [ ]:
!fairseq-preprocess --destdir data/data_norm_bin_2000/ \
                    -s trg -t src \
                    --trainpref data/train.sp2000 \
                    --validpref data/dev.sp2000 \
                    --testpref data/test.sp2000 \
                    --joined-dictionary

Maintenant on peut appeler fairseq-train avec les paramètres souhaités

- Le modèle entraîné se trouvera dans `models/new_norm_lstm/` (l'option `--save-dir`).
- En pratique, le modèle sera sauvegardé plusieurs fois pendant l'entraînement (des checkpoints). La fréquence de sauvegarde peut être choisi avec l'option `--save-interval` (tous les `n` epochs)

In [ ]:
# create an empty model folder to store the model in
!mkdir models/new_norm_lstm

# call fairseq-train
!fairseq-train \
        data/data_norm_bin_2000 \
        --save-dir models/new_norm_lstm \
        --save-interval 1 --patience 12 \
        --arch lstm \
        --encoder-layers 3 --decoder-layers 3 \
        --encoder-embed-dim 384 --decoder-embed-dim 384 --decoder-out-embed-dim 384 \
        --encoder-hidden-size 768 --encoder-bidirectional --decoder-hidden-size 768 \
        --dropout 0.3 \
        --criterion cross_entropy --optimizer adam --adam-betas '(0.9, 0.98)' \
        --lr 0.0001 --lr-scheduler inverse_sqrt \
        --warmup-updates 4000 \
        --share-all-embeddings \
        --max-tokens 3000 \
        --batch-size-valid 64

### Entraîner un modèle de type transformer

In [ ]:
# create an empty model folder to store the model in
!mkdir models/new_norm_lstm

# call fairseq-train
!fairseq-train \
        data/data_norm_bin_2000 \
        --save-dir models/new_norm_transformer \
        --save-interval 1 --patience 25 \
        --arch transformer \
        --encoder-layers 2 --decoder-layers 4 --encoder-attention-heads 4 \
        --encoder-embed-dim 256 --encoder-ffn-embed-dim 1024 --dropout 0.3 \
        --criterion cross_entropy --optimizer adam --adam-betas '(0.9, 0.98)' \
        --lr 0.001 --lr-scheduler inverse_sqrt \
        --warmup-updates 4000 \
        --max-tokens 3000 --max-tokens 3000 \
        --share-all-embeddings --batch-size-valid 64